In [29]:
pip install autocorrect

Note: you may need to restart the kernel to use updated packages.


In [30]:
import typing
import re
from typing import NamedTuple
from collections import defaultdict
from nltk.corpus import stopwords
import math 
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
import nltk

In [32]:
from autocorrect import spell

In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RICARDO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\RICARDO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
class message(NamedTuple):
    text: str
    is_spam:bool

Se desea pronosticar con una probabilidad bayessiana P(spam|token), por lo cual aplicando el teorema obtenemos P(token|spam) y P(token|Nospam)

# Contrucción del modelo

In [36]:
class NaiveBayesClassifier:
    
    def __init__(self,k=0.5):
        self.k=k
        self.tokens=set()# crea un conjunto, que es una coleccion desordena de elementos unicos
        self.token_spam_counts=defaultdict(int)#crea diccionarios que tienen claves con valor de cero automatico, su valor cuenta el numero de veces que una palabras esta registra como spam
        self.token_ham_counts=defaultdict(int)#el valor de este diccionario cuenta el numero de veces que una palabra se registra como no spam o ham
        self.spam_messages=self.ham_messages=0# se inician en cero el nuemro de mensajes de spam y de no spam
    
    def tokenize(self, text):
        text=text.lower()
        all_words=re.findall("[a-z0-9]+",text)#extrae palabras
        return set(all_words)#retorna una coleccion donde cada elmento es una palabra
        
    def fit(self,x_train,y_train):
        #recorre cada uno de los registros de los datos de entrenamiento
        for i in range(0,y_train.shape[0]):
            #incrementa el conteo de si es es spam o no
            if y_train[i]==1:
                self.spam_messages+=1# cuenta registros de spam
            else:
                self.ham_messages+=1 #cuenta registros que no son spam
            
            # incremento de conteo de palabras en el diccionario si pertenecen a spam o no
            for token in self.tokenize(x_train[i]):#retorna una coleccion con todas las plabras sin duplciados ej {'ciencia','datos'}
                self.tokens.add(token)#añade cada palabra a la coleccion sin importar si es spam o no
                
                if y_train[i]==1:#revisa si el mensaje en que esta iterando el bucle es spam se se añade a su respectivo diccionario y aumentan la cuenta de sur valor
                    self.token_spam_counts[token]+=1 #cuenta tokens de spam
                else:
                    self.token_ham_counts[token]+=1 #cuenta tokens que no son spam
                    
     #obtiene las probabilidaes condicionadas a partir de los datos de entrenamiento               
    def probabilities(self,token):
        spam=self.token_spam_counts[token]#le asigna el valor de conteo que ha tenido un token considerado spam a partir del entramiento
        ham=self.token_ham_counts[token]#si no existe asigna un cero
        #P(token|spam) 
        p_token_spam=(spam+self.k)/(self.spam_messages+(2*self.k))# es un parametro en donde se asumen k palabras adicionales de spam y 2k mensajes de spam adicionales  
        #P(token|ham)
        p_token_ham=(ham+self.k)/(self.ham_messages+(2*self.k))
        
        return p_token_ham,p_token_spam
    
    #predice la probabilidad
    def predict_prob(self, x_test):
        result=[]
        for i in range(x_test.shape[0]):
            text_tokens=self.tokenize(x_test[i]) #obtiene la coleccion de palabras
            log_prob_if_spam=log_prob_if_ham=0.0
            
            #iterar cada palabra en el vocabulario y asi se van sumando la probabildiaddes logaritmoicas que seria igual que multippicar p(token1|spam)*p(token_i|spam)
            for token in self.tokens:
                prob_if_ham,prob_if_spam=self.probabilities(token)#obtiene las probabiidad p(token|spam) p(token|ham)
                
                if token in text_tokens:
                    #se suman las probabilidades logaritmicas de cada palabra de un registro dado un evento para el onjunto de prueba
                    log_prob_if_spam+=math.log(prob_if_spam)
                    log_prob_if_ham+=math.log(prob_if_ham)
                
                else:
                    log_prob_if_spam+=math.log(1.0-prob_if_spam)
                    log_prob_if_ham+=math.log(1.0-prob_if_ham)

            #ahora sacamos el exponencial del logaritmo para obtener p(token1|spam)*p(token_n|spam)
            prob_if_spam=math.exp(log_prob_if_spam)
            prob_if_ham=math.exp(log_prob_if_ham)
            
            #aca agregamos la probabilidad de que una muestra sea spam o no
            try:
                result.append(prob_if_spam/(prob_if_spam+prob_if_ham))
            except:
                print('Introduzca mas muestras para entrenamiento')
                result.append(0.5)
        #ahora retornarmos la probabilidad de que sea spam dado que hay spam y no spam
        return np.array(result)
         
    #predice la etiqueta
    def predict(self,x_test):
        label=np.where(self.predict_prob(x_test)>0.5,1,0)
        return label

In [37]:
def tokenize(text):
    text=text.lower()
    all_words=re.findall("[a-z0-9]+",text)#extrae palabras
    return set(all_words)#retorna una coleccion donde cada elmento es una palabra

# Preprocesamiento de los datos

In [38]:
datos=pd.read_csv('D:\Documentos\Aprendizaje\PHYTON\Data Science\spam.csv',encoding='latin-1')
datos.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [39]:
datos=datos.rename(columns={'v1':'is_spam','v2':'text'})

In [40]:
datos.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [41]:
fact=pd.factorize(datos['is_spam'])
datos['is_spam']=fact[0]

In [42]:
datos['count']=datos['text'].apply(lambda x: len(str(x).split(" ")))

In [43]:
stopwords=stopwords.words('english')

In [44]:
def limpieza(texto):
    texto = re.sub(r"https?\://\S+", '', texto)
    texto = re.sub(r'\<a href', ' ', texto)
    texto = re.sub(r'&amp;', 'and', texto)
    texto = re.sub(r'<br />', ' ', texto)
    texto = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', texto)
    texto = re.sub('\d','', texto)
    texto = re.sub(r"can\'t", "cannot", texto)
    texto = re.sub(r"it\'s", "it is", texto)
    return texto

In [45]:
datos['text']=datos['text'].apply(lambda x: limpieza(str(x)))

In [46]:
datos['stop_count']=datos['text'].apply(lambda x: len([x for x in str(x).split(" ") if x in stopwords]))
datos['text']=datos['text'].apply(lambda x: " ".join(x.lower() for x in str(x).split())) 
datos['text']=datos['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stopwords))
#datos['text']=[' '.join([spell(i) for i in x.split()]) for x in datos['text']]

In [47]:
especial=r"[^a-z0-9\s]"#borrar los caracteres que no este especificados por eso el ^
datos['text']=datos['text'].str.replace(especial," ")

C:\Users\RICARDO\AppData\Local\Temp\ipykernel_10776\722356309.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  datos['text']=datos['text'].str.replace(especial," ")


In [48]:
lema=nltk.stem.WordNetLemmatizer()

In [49]:
datos['text']=datos['text'].apply(lambda x: " ".join([lema.lemmatize(x) for x in x.split()]))

In [50]:
datos=datos[['text','is_spam']]

# Partición de los datos

In [51]:
datos.head()

,text,is_spam
0,go jurong point crazy available bugis n great ...,0
1,ok lar joking wif u oni,0
2,free entry wkly comp win fa cup final tkts st ...,1
3,u dun say early hor u c already say,0
4,nah think go usf life around though,0


In [52]:
x_train,y_train=np.array(datos.iloc[0:(int(0.8*3000)),0]),np.array(datos.iloc[0:(int(0.8*3000)),1])


In [53]:
x_test,y_test=np.array(datos.iloc[int(0.8*3000):,0]),np.array(datos.iloc[int(0.8*3000):,1])

# Entrenamiento del modelo

In [54]:
bayes=NaiveBayesClassifier()

In [55]:
bayes.fit(x_train,y_train)

# Prueba del modelo

In [56]:
resultados=bayes.predict(x_test)

In [60]:
print(classification_report(y_test,resultados))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2760
           1       0.98      0.86      0.92       412

    accuracy                           0.98      3172
   macro avg       0.98      0.93      0.95      3172
weighted avg       0.98      0.98      0.98      3172

